In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

Un RDD (Resilient Distributed Dataset) es la estructura básica de datos en Spark.
Es una colección de datos distribuida en varios nodos de un cluster y permite realizar operaciones en paralelo.

In [49]:
# Crea un contexto de Spark , para hacer la conexion entre mi codigo y el motor de Spark
#master = 'local' -> significa que spark se ejecutara en la maquina local usando solo nucleo de CPU 'local[*]' para usar todos los nucleos disponibles ,, En un entorno distribuido, podrías usar algo como 'yarn' o 'spark://<host>:<port>'.
# El parámetro appName define el nombre de la aplicación en Spark, útil para monitorear el trabajo en la interfaz web de Spark UI.
sc = SparkContext(master = 'local',appName =  'TransformacionesAcciones')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=DataFrameExample, master=local[*]) created by getOrCreate at <ipython-input-6-ca4da2a712d8>:2 

In [ ]:
# Create a SparkSession
spark = SparkSession.builder.appName("DataFrameExample").getOrCreate()

In [ ]:
data = [1,2,3,4]
rdd = sc.parallelize(data)

In [ ]:
type(rdd)

pyspark.rdd.RDD

In [ ]:
rdd.collect()

[1, 2, 3, 4]

In [ ]:
sc

<SparkContext master=local appName=TransformacionesAcciones>

In [8]:
path = '/content/drive/MyDrive/Spark/files/'

# Obtencion csv con spark Cntext

In [ ]:
# Obtyencion de archivo csv con spark Context
equiposOlimpicosRDD = sc.textFile(path + 'paises.csv').map(lambda line : line.split(","))

In [ ]:
equiposOlimpicosRDD.take(10)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

In [ ]:
 # ver cuantos paises participaron
equiposOlimpicosRDD.map(lambda x : (x[2])).distinct().count()# x[2] el elemnto 2 de cada lista


231

In [ ]:
#Cuenta los rdd dentro de los primeros 20 milisegundos
equiposOlimpicosRDD.countApprox(20)

1185

In [ ]:
# ver los primeros elementos para ver el campo en comun entre equiposRDD y deportistaRDD
equiposOlimpicosRDD.show(2)

In [ ]:
#Importamos los csv deportista y deportista2
deportistaRDD = sc.textFile(path + 'deportista.csv').map(lambda line : line.split(","))
deportista2RDD = sc.textFile(path + 'deportista2.csv').map(lambda line : line.split(","))

AttributeError: 'SparkSession' object has no attribute 'textFile'

In [ ]:
# Hacemos la union de los dos rdd
deportistaRDD = deportistaRDD.union(deportista2RDD)

In [ ]:
#contamos la cantidad de rdds
deportistaRDD.count()

In [ ]:
deportistaRDD.show()


## Obtener equipo y deportista

In [ ]:
# es id de equiposRDD y equipo_id de deportistsRDD lo que los une
# Se selecciona la última columna del RDD (equipo_id) que es valor eje y el resto de contenidos
 # Se selecciona solo el id que es el valor eje y la sigla del país


deportistasEquipos = deportistaRDD.map(lambda line : [line[-1],line[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]]))#.takeSample(False,6,25)  #False si quiero que no se repitan , 6 cuantos quiero en la salida  y 25 la semilla de aleatoriedad

In [ ]:
lista_deportisata_equipo = deportistaRDD.map(lambda line : [line[-1],line[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])).top(10)


In [ ]:
lista_deportisata_equipo[:]

[('999', (['92679', 'Trygve Bjarne Pedersen', '1', '35', '0', '0'], 'NOR')),
 ('999', (['1144', 'Henrik Agersborg', '1', '47', '0', '0'], 'NOR')),
 ('999', (['10765', 'Einar Berntsen', '1', '28', '0', '0'], 'NOR')),
 ('998',
  (['111659', 'G. Bernard Bernie Skinner', '1', '34', '182', '82'], 'CAN')),
 ('996', (['116030', 'Edward Eddy Stutterheim', '1', '39', '0', '0'], 'NED')),
 ('995', (['71728', 'Gordon Frederick Love', '1', '26', '0', '0'], 'FRA')),
 ('995', (['101553', 'Louis Roche', '1', '0', '0', '0'], 'FRA')),
 ('994', (['87856', 'Leo Anthony O Connell', '1', '20', '0', '0'], 'USA')),
 ('994', (['55319', 'Johnson', '1', '0', '0', '0'], 'USA')),
 ('994', (['53520', 'Henry Wood Jameson', '1', '21', '0', '0'], 'USA'))]

In [ ]:
lista_deportisata_equipo[1][0]

'999'

In [ ]:
lista_deportisata_equipo[1][0][0]

'9'

In [ ]:
deportistasEquipos.map(lambda x : (x[1][0][0],x[1][0][1:],x[1][1])).top(2)

[('99999', ['Alexander Grant Alick Rennie', '1', '32', '182', '71'], 'RSA'),
 ('99998', ['Robert John Bob Renney', '1', '21', '178', '90'], 'AUS')]

In [ ]:
# obtenemos info de resultado
resultado = sc.textFile(path + 'resultados.csv').map(lambda line : line.split(","))

In [ ]:
resultado.take(7)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1'],
 ['2', 'NA', '2', '49', '2'],
 ['3', 'NA', '3', '7', '3'],
 ['4', 'Gold', '4', '2', '4'],
 ['5', 'NA', '5', '36', '5'],
 ['6', 'NA', '5', '36', '6']]

In [ ]:
#Filtrar solo a los que ganaron medalla
resultado = resultado.filter(lambda l : 'NA' not in l[1])

In [ ]:
resultado.take(3)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19']]

In [ ]:
deportistaRDD.top(2)



[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [ ]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [ ]:
# Juntamos el RDD de deportistas ,equiposcon el de resultado

# 1er parte juntamos deportistas con y sus equipos
deportistas_Equipos = deportistaRDD.map(lambda line : [line[-1],line[:-1]]).join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]]))


In [ ]:
deportistas_Equipos.takeSample(False,6,25)

[('308', (['117713', 'Ahmed Amin Tabouzada', '1', '0', '0', '0'], 'EGY')),
 ('96', (['95174', 'Dsir Piryns', '1', '0', '0', '0'], 'BEL')),
 ('1178', (['99228', 'Viktor Ravnik', '1', '22', '179', '80'], 'YUG')),
 ('888', (['52248', 'Vaipava Nevo Ioane', '1', '28', '152', '62'], 'SAM')),
 ('716', (['108017', 'Neville Ian Scott', '1', '21', '187', '74'], 'NZL')),
 ('1019', (['39331', 'Werner Geeser', '1', '23', '182', '68'], 'SUI'))]

In [ ]:
deportistas_Equipos.map(lambda x  : (x[1][0][0],(x[1][0][1:],x[1][1]))).take(2)

[('1', (['A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('2', (['A Lamusi', '1', '23', '170', '60'], 'CHN'))]

In [ ]:
# 2da parte Juntamos con los resultados obteniendo las medallas de cada deportista haciendo la union con el deportista_id
deportistasGanadores = deportistas_Equipos.map(lambda x  : (x[1][0][0],(x[1][0][1:],x[1][1]))).join(resultado.map(lambda y : (y[2],y[1])))

In [ ]:
deportistasGanadores.take(2)

[('17996', ((['Cao Mianying', '2', '21', '176', '71'], 'CHN'), 'Silver')),
 ('18005', ((['Cao Yuan', '1', '17', '160', '42'], 'CHN'), 'Gold'))]

Obtenemos los puntajes de cada pais de acuerdo al valor de las medallas

In [ ]:
# Diccionario valores de las medallas
valoresMedallas = {'Gold':7,'Silver':5,'Bronze':4}

In [ ]:
# primer parte se reduce la tupla
# x[1][0][1] -> Iniciales pais
# x[1][1] -> tipo de medalla
deportistasGanadores.map(lambda x : (x[1][0][1],x[1][1])).take(2)

[('CHN', 'Silver'), ('CHN', 'Gold')]

In [ ]:
# Segunda parte obtenemos los valores de cada medalla
deportistasGanadores.map(lambda x : (x[1][0][1],valoresMedallas[x[1][1]])).take(2)

[('CHN', 5), ('CHN', 7)]

In [ ]:
# Guardamos en una  variable el rdd donde esta pais , puntos por medalla
paisesPuntos = deportistasGanadores.map(lambda x : (x[1][0][1],valoresMedallas[x[1][1]]))

In [ ]:
paisesPuntos.take(2)

[('CHN', 5), ('CHN', 7)]

In [ ]:
# hacemos la sumatoria por pais y ordenamos los resultados
from operator import add
conclusion = paisesPuntos.reduceByKey((add)).sortBy(lambda x : x[1],ascending = False)


In [ ]:
conclusion.take(10)

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142)]

# Obtencion de archivos csv con Spark Session

In [12]:
#Obtencion de archivo csv con spark session , cuando se hace la lectura con spak session se transforma en dataframe y con context en rdd
deportistaOlimpicoDF= spark.read.csv(path + 'deportista.csv',header = True)
deportistaOlimpico2DF= spark.read.csv(path + 'deportista2.csv',header = True)

In [13]:
# Union de los 2 Data Frame
DeportistaDF = deportistaOlimpicoDF.union(deportistaOlimpico2DF)

# Dataframes

In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
from pyspark.sql.types import Row

In [5]:
path = '/content/drive/MyDrive/Spark/files'

Creacion de schema

In [ ]:
juegosSchema = StructType([StructField('juego_id',IntegerType(),False),# False si es obligatorio
                           StructField('nombre_juego',StringType(),False),
                           StructField("año",StringType(),False),
                           StructField("temporada",StringType(),False),
                           StructField("ciudad",StringType(),False)])

Lectura de schema

In [6]:
# Create a SparkSession
spark = SparkSession.builder.appName("DataFrameExample").getOrCreate()


Transformacion de RDD -> Df

In [7]:
# "header", "true" → Le dice a Spark que la primera fila del CSV contiene los nombres de las columnas.
juegoDF = spark.read.schema(juegosSchema).option ("header","true").csv(path + '/juegos.csv')

NameError: name 'juegosSchema' is not defined

In [ ]:
juegoDF.show(5)

+--------+------------+----+---------+---------+
|juego_id|nombre_juego| año|temporada|   ciudad|
+--------+------------+----+---------+---------+
|       1| 1896 Verano|1896|   Verano|   Athina|
|       2| 1900 Verano|1900|   Verano|    Paris|
|       3| 1904 Verano|1904|   Verano|St. Louis|
|       4| 1906 Verano|1906|   Verano|   Athina|
|       5| 1908 Verano|1908|   Verano|   London|
+--------+------------+----+---------+---------+
only showing top 5 rows



In [ ]:
spark

In [ ]:
deportistaRDD.show()

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|  80|      199|
|            2|            A Lamusi|     1|  23|   170|  60|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0|   0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0|   0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|  82|      705|
|            6|     Per Knut Aaland|     1|  31|   188|  75|     1096|
|            7|        John Aalberg|     1|  31|   183|  72|     1096|
|            8|Cornelia Cor Aalt...|     2|  18|   168|   0|      705|
|            9|    Antti Sami Aalto|     1|  26|   186|  96|      350|
|           10|Einar Ferdinand E...|     1|  26|     0|   0|      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182|76.5|      350|
|     

In [ ]:
type(deportistaRDD)

pyspark.sql.dataframe.DataFrame

In [14]:
# Cambiar los tipos de datos del dataframe DeportistaDF
dict_tipos = {'deportista_id':IntegerType(),'genero':IntegerType(),'edad':IntegerType(),'altura':IntegerType(),'peso':FloatType(),'equipo_id':IntegerType()}

for col,tipo in dict_tipos.items():
  DeportistaDF = DeportistaDF.withColumn(col,DeportistaDF[col].cast(tipo))


In [15]:
DeportistaDF.dtypes

[('deportista_id', 'int'),
 ('nombre', 'string'),
 ('genero', 'int'),
 ('edad', 'int'),
 ('altura', 'int'),
 ('peso', 'float'),
 ('equipo_id', 'int')]

In [ ]:
#Eliminamos columna Altura
DeportistaDF = deportistaRDD.drop('altura')

In [16]:
# Reasignamos nombre de columnas
DeportistaDF = DeportistaDF.withColumnRenamed('genero','sexo')

In [17]:
DeportistaDF.columns

['deportista_id', 'nombre', 'sexo', 'edad', 'altura', 'peso', 'equipo_id']

In [ ]:
# Uso de select
DeportistaDF.select('nombre','edad').show(5)

+--------------------+----+
|              nombre|edad|
+--------------------+----+
|           A Dijiang|  24|
|            A Lamusi|  23|
| Gunnar Nielsen Aaby|  24|
|Edgar Lindenau Aabye|  34|
|Christine Jacoba ...|  21|
+--------------------+----+
only showing top 5 rows



In [18]:
import pyspark.sql.functions as f

#Si usas col(), puedes encadenar métodos y hacer transformaciones más complejas,

DeportistaDF = DeportistaDF.select('deportista_id','nombre','sexo',f.col('edad').alias('edadAlJugar'),'equipo_id')

In [ ]:
DeportistaDF

DataFrame[deportista_id: int, nombre: string, edadAlJugar: int, equipo_id: int]

In [ ]:
#Ordenar el Dataframe segun una columna
DeportistaDF.sort('edadAlJugar').show()

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        68312|      Beyene Legesse|          0|      332|
|          224|     Mohamed AbdelEl|          0|      308|
|        68652|    Adjutant Lemoine|          0|      362|
|          487|      Inni Aboubacar|          0|      721|
|        68315|     Douglas C. Legg|          0|      413|
|          226|Sanad Bushara Abd...|          0|     1003|
|        68302|         Viktor Legt|          0|      259|
|           58|    Georgi Abadzhiev|          0|      154|
|        68317|      Ronald H. Legg|          0|      413|
|          230|    Moustafa Abdelal|          0|      308|
|        67849|          Edward Lee|          0|      199|
|          102|   Sayed Fahmy Abaza|          0|      308|
|        68329|        Paul Legrain|          0|      362|
|          260|  Ahmed Abdo Mustafa|          0|     100

In [19]:
#Filtramos los deportistas que sean diferentes de cero
DeportistaDF = DeportistaDF.filter(DeportistaDF.edadAlJugar != 0)

In [20]:
DeportistaDF.sort('edadAlJugar').show()

+-------------+--------------------+----+-----------+---------+
|deportista_id|              nombre|sexo|edadAlJugar|equipo_id|
+-------------+--------------------+----+-----------+---------+
|        71691|  Dimitrios Loundras|   1|         10|      333|
|        22411|Magdalena Cecilia...|   2|         11|      413|
|        70616|          Liu Luyang|   2|         11|      199|
|        37333|Carlos Bienvenido...|   1|         11|      982|
|        76675|   Marcelle Matthews|   2|         11|      967|
|        40129|    Luigina Giavotti|   2|         11|      507|
|       118925|Megan Olwen Deven...|   2|         11|      413|
|        47618|Sonja Henie Toppi...|   2|         11|      742|
|       126307|        Liana Vicens|   2|         11|      825|
|        51268|      Beatrice Hutiu|   2|         11|      861|
|        52070|        Etsuko Inada|   2|         11|      514|
|        72854|      Licia Macchini|   2|         12|      507|
|         5291|Marcia Arriaga La...|   2

In [21]:
#Obtencion de paises
paises = spark.read.csv(path + 'paises.csv',header=True)

In [ ]:
paises.dtypes

[('id', 'string'), ('equipo', 'string'), ('sigla', 'string')]

In [22]:
paises = paises.withColumn('id',paises['id'].cast(IntegerType()))

In [ ]:
paises.show(5)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



In [23]:
# Obtencion de evento
eventoDF = spark.read.csv(path + 'evento.csv',header=True)

In [24]:
eventoDF.show(3)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
+---------+--------------------+----------+
only showing top 3 rows



In [25]:
eventoDF.dtypes

[('evento_id', 'string'), ('evento', 'string'), ('deporte_id', 'string')]

In [26]:
eventoDF = eventoDF.withColumn('evento_id',eventoDF['evento_id'].cast(IntegerType()))


In [27]:
#Obtewncion de resultados
resultados = spark.read.csv(path + 'resultados.csv',header=True)

In [28]:
resultados.dtypes

[('resultado_id', 'string'),
 ('medalla', 'string'),
 ('deportista_id', 'string'),
 ('juego_id', 'string'),
 ('evento_id', 'string')]

In [29]:
dict = {'resultado_id':IntegerType(),'medalla':StringType(),'deportista_id':IntegerType(),'juego_id':IntegerType(),'evento_id':IntegerType()}

for clave,valor in dict.items():
  resultados = resultados.withColumn(clave,resultados[clave].cast(valor))

In [30]:
resultados.dtypes

[('resultado_id', 'int'),
 ('medalla', 'string'),
 ('deportista_id', 'int'),
 ('juego_id', 'int'),
 ('evento_id', 'int')]

In [ ]:
resultados.printSchema()

root
 |-- resultado_id: integer (nullable = true)
 |-- medalla: string (nullable = true)
 |-- deportista_id: integer (nullable = true)
 |-- juego_id: integer (nullable = true)
 |-- evento_id: integer (nullable = true)



In [31]:
#Obtencion de juegos
juegosDF = spark.read.csv(path+'/juegos.csv',header=True)

In [32]:
#Renombramos y cambiamos tipo de datos
juegosDF = juegosDF.select(f.col('_c0').alias('juego_id').cast(IntegerType()),'nombre_juego','annio','temporada','ciudad')

In [ ]:
juegosDF.show(5)

+--------+------------+-----+---------+---------+
|juego_id|nombre_juego|annio|temporada|   ciudad|
+--------+------------+-----+---------+---------+
|       1| 1896 Verano| 1896|   Verano|   Athina|
|       2| 1900 Verano| 1900|   Verano|    Paris|
|       3| 1904 Verano| 1904|   Verano|St. Louis|
|       4| 1906 Verano| 1906|   Verano|   Athina|
|       5| 1908 Verano| 1908|   Verano|   London|
+--------+------------+-----+---------+---------+
only showing top 5 rows



In [33]:
juegosDF = juegosDF.withColumn('annio',juegosDF['annio'].cast(IntegerType()))

In [ ]:
juegosDF.dtypes

[('juego_id', 'int'),
 ('nombre_juego', 'string'),
 ('annio', 'int'),
 ('temporada', 'string'),
 ('ciudad', 'string')]

In [ ]:
# Uso de joins para unir data frames
DeportistaDF.join( resultados,DeportistaDF.deportista_id == resultados.deportista_id , 'left')\
.join(juegosDF, juegosDF.juego_id == resultados.juego_id,'left')\
.join(eventoDF,eventoDF.evento_id == resultados.evento_id,'left')\
.select('nombre','edadAlJugar','medalla',f.col('annio').alias('año'),'evento').filter(resultados.medalla!= 'NA').show(5)

+--------------------+-----------+-------+----+--------------------+
|              nombre|edadAlJugar|medalla| año|              evento|
+--------------------+-----------+-------+----+--------------------+
|Edgar Lindenau Aabye|         34|   Gold|1900|Tug-Of-War Men's ...|
|Arvo Ossian Aaltonen|         22| Bronze|1920|Swimming Men's 20...|
|Arvo Ossian Aaltonen|         22| Bronze|1920|Swimming Men's 40...|
|Juhamatti Tapio A...|         28| Bronze|2014|Ice Hockey Men's ...|
|Paavo Johannes Aa...|         28| Bronze|1948|Gymnastics Men's ...|
+--------------------+-----------+-------+----+--------------------+
only showing top 5 rows



In [ ]:
# union de medalla , pais y equipo que pertenecen
DeportistaDF.join(resultados,DeportistaDF.deportista_id == resultados.deportista_id,'left')\
.join(paises,paises.id == DeportistaDF.equipo_id,'left')\
.select('medalla','equipo','sigla').filter(resultados.medalla !='NA').show(5)

+-------+--------------+-----+
|medalla|        equipo|sigla|
+-------+--------------+-----+
|   Gold|Denmark/Sweden|  SWE|
| Bronze|       Finland|  FIN|
| Bronze|       Finland|  FIN|
| Bronze|       Finland|  FIN|
| Bronze|       Finland|  FIN|
+-------+--------------+-----+
only showing top 5 rows



In [35]:
medallistaXAnio = DeportistaDF.join(resultados,DeportistaDF.deportista_id == resultados.deportista_id,'left')\
.join(juegosDF,juegosDF.juego_id == resultados.juego_id,'left')\
.join(paises,paises.id == DeportistaDF.equipo_id,'left')\
.join(eventoDF,eventoDF.evento_id == resultados.evento_id,'left')\
.select('sigla',f.col('annio').alias('año'),'medalla','equipo','nombre','evento')\
.filter(resultados.medalla !='NA')

In [39]:
medallista2XAnio = medallistaXAnio.groupBy('sigla','año').count()

In [40]:
medallista2XAnio.show()

+-----+----+-----+
|sigla| año|count|
+-----+----+-----+
|  USA|2012|  248|
|  BLR|2000|   15|
|  FRA|2006|   15|
|  FIN|1988|   38|
|  KOR|2010|   18|
|  VEN|2012|    1|
|  FRA|1948|   77|
|  GBR|2000|   55|
|  QAT|2012|    2|
|  FRG|1994|    6|
|  JPN|1932|   28|
|  KOR|1972|    1|
|  GER|1932|   57|
|  AUS|1972|   20|
|  NED|1972|   15|
|  BAH|2008|    7|
|  NZL|1988|   24|
|  THA|1988|    1|
|  POR|1988|    1|
|  POR|2012|    2|
+-----+----+-----+
only showing top 20 rows



In [41]:
medallista2XAnio.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- año: integer (nullable = true)
 |-- count: long (nullable = false)



In [42]:
# Agrupamps por medallas totales y medallas promedio
medallista2XAnio.groupBy('sigla','año').agg(f.sum('count').alias('Total medallas'),f.avg('count').alias('Medallas promedio')).sort('año').show()

+-----+----+--------------+-----------------+
|sigla| año|Total medallas|Medallas promedio|
+-----+----+--------------+-----------------+
|  DEN|1896|             6|              6.0|
|  FRA|1896|            11|             11.0|
|  AUT|1896|             5|              5.0|
|  HUN|1896|             6|              6.0|
|  GER|1896|            30|             30.0|
|  AUS|1896|             3|              3.0|
|  GBR|1896|             8|              8.0|
|  USA|1896|            20|             20.0|
|  GRE|1896|             9|              9.0|
|  SUI|1896|             3|              3.0|
|  IND|1900|             2|              2.0|
|  AUS|1900|             6|              6.0|
|  CAN|1900|             2|              2.0|
|  ITA|1900|             5|              5.0|
|  ESP|1900|             2|              2.0|
|  AUT|1900|             6|              6.0|
|  SUI|1900|            21|             21.0|
|  DEN|1900|             7|              7.0|
|  CUB|1900|             2|       

#SQL

In [44]:
resultados.registerTempTable('resultado')
DeportistaDF.registerTempTable('deportista')
paises.registerTempTable('paises')
juegosDF.registerTempTable('juegos')
eventoDF.registerTempTable('evento')

In [57]:

spark.sql("SELECT * FROM resultado").show(5)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



In [61]:
#Requiere mas procesamiento al usar sql
spark.sql(""" SELECT medalla,equipo,sigla,j.annio FROM resultado r
              JOIN deportista d
              ON r.deportista_id = d.deportista_id
              JOIN paises p
              ON p.id = d.equipo_id
              JOIN juegos j
              ON j.juego_id = r.juego_id
              JOIN evento e
              ON e.evento_id = r.evento_id
              WHERE medalla <> "NA"
              GROUP BY medalla,equipo,sigla,j.annio
              ORDER BY annio ASC""").show()

+-------+--------------------+-----+-----+
|medalla|              equipo|sigla|annio|
+-------+--------------------+-----+-----+
| Silver|              France|  FRA| 1896|
|   Gold|       United States|  USA| 1896|
|   Gold|              Greece|  GRE| 1896|
|   Gold|           Australia|  AUS| 1896|
| Bronze|             Germany|  GER| 1896|
|   Gold|             Denmark|  DEN| 1896|
| Silver|       Great Britain|  GBR| 1896|
| Silver|       United States|  USA| 1896|
| Silver|             Denmark|  DEN| 1896|
| Silver|             Austria|  AUT| 1896|
| Bronze|Ethnikos Gymnasti...|  GRE| 1896|
| Bronze|       United States|  USA| 1896|
|   Gold|             Austria|  AUT| 1896|
| Bronze|             Denmark|  DEN| 1896|
| Bronze|       Great Britain|  GBR| 1896|
| Bronze|              France|  FRA| 1896|
|   Gold|             Hungary|  HUN| 1896|
|   Gold|       Great Britain|  GBR| 1896|
|   Gold|              France|  FRA| 1896|
| Bronze|             Hungary|  HUN| 1896|
+-------+--

# UDF
Las funciones definidas por el usuario o UDF, por sus siglas en inglés, son una funcionalidad agregada en Spark para definir funciones basadas en columnas las cuales permiten extender las capacidades de Spark al momento de transformar el set de datos.

Este tipo de implementaciones son convenientes cuando tenemos un desarrollo extenso donde hemos identificado la periodicidad de tareas repetitivas como suele ser en pasos de limpieza de datos, transformación o renombrado dinámico de columnas.

Por lo anterior es común encontrar en un proyecto de Spark una librería independiente donde existen todas estas funciones agregadas para que los desarrolladores involucrados en el proyecto puedan usarlas a conveniencia.

El uso de UDF no implica que las funciones que podemos crear nativamente con Python, Scala, R o Java no sean útiles. Una UDF tiene el objetivo de ofrecer un estándar interno en el proyecto que nos encontremos realizando. Además, en caso de ser necesario, una UDF puede ser modificada con ayuda de decoradores para que sea más extensible en diversos escenarios a los cuales nos podemos enfrentar.

Otro motivo para usar UDF es que en el módulo de Spark MLlib, la librería nativa de Spark para operaciones de Machine Learning, las UDF juegan un papel vital al momento de hacer transformaciones. Por lo cual tener un uso familiar de estas ampliará considerablemente la curva de aprendizaje de Spark MLlib.

In [62]:
spark.stop()

In [ ]:
sc.stop()